<a href="https://colab.research.google.com/github/Teja21B21A4206/fmml-labs1/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [2]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [3]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [4]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [5]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [6]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [7]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [8]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [9]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [10]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [11]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [12]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [13]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [14]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


1. Does averaging the validation accuracy across multiple splits give more consistent results?

A.Yes, averaging the validation accuracy across multiple splits of your dataset can often give you more consistent and reliable results when assessing the performance of a machine learning model. This technique is commonly known as cross-validation.

Cross-validation helps address the potential issue of the randomness in how data is split into training and validation sets. When you split your data into a single training and validation set, the specific data points in each set can greatly affect the performance metric (e.g., accuracy) you obtain. This variability may not reflect the true generalization ability of your model.

Cross-validation works by splitting your dataset into multiple (often k) subsets or folds. You train and validate your model k times, each time using a different fold as the validation set and the remaining folds as the training set. You then average the performance metrics (e.g., accuracy) obtained across these k iterations.

Here are some advantages of using cross-validation:

Reduced Variance: By averaging results over multiple splits, cross-validation reduces the variance in your performance estimates, providing a more stable and representative evaluation of your model's performance.

Better Generalization Assessment: Cross-validation provides a more accurate assessment of how well your model generalizes to new, unseen data because it tests the model on multiple different subsets of the data.

Efficient Use of Data: It allows you to make better use of your data because all data points are used for both training and validation across the k iterations.

However, keep in mind that cross-validation can be computationally expensive, especially when working with large datasets or complex models. It's also important to choose an appropriate value of k (e.g., 5-fold or 10-fold cross-validation) based on the size of your dataset and computational resources.

In summary, averaging validation accuracy (or other performance metrics) across multiple splits using cross-validation is a common and valuable technique for obtaining more consistent and reliable assessments of your machine learning model's performance.







2.Does it give more accurate estimate of test accuracy?


A.Cross-validation, which involves averaging validation accuracy across multiple splits, provides a more accurate estimate of how well your model is likely to perform on unseen data compared to a single train-test split. However, it's essential to understand that cross-validation estimates are still not a direct replacement for the true test accuracy.

Here's why cross-validation provides a more accurate estimate of test accuracy:

Reduces Variance: Cross-validation helps reduce the variability in performance estimates. With a single train-test split, the specific data points in the test set can significantly impact the test accuracy. Cross-validation averages out these variations across multiple splits, resulting in a more stable and reliable performance estimate.

Better Generalization Assessment: Cross-validation tests your model's performance on multiple subsets of the data, which mimics the scenario of evaluating the model on different unseen datasets. This gives you a more comprehensive view of how well your model generalizes to new, unseen data.

Efficient Use of Data: Cross-validation ensures that all data points are used for both training and validation, which can lead to a more representative evaluation of your model's capabilities.

Despite these advantages, it's important to note that cross-validation estimates are still not the same as the true test accuracy. The true test accuracy is obtained by evaluating your model on a completely separate and unseen dataset that was not used during model development or hyperparameter tuning. This dataset serves as a final, unbiased evaluation of your model's performance.

In practice, cross-validation is often used for model selection, hyperparameter tuning, and getting a robust estimate of how well a model is likely to perform on unseen data. Once you have a well-tuned model, you should still reserve a separate holdout dataset (a true test set) to obtain the final, unbiased estimate of your model's performance.

So, while cross-validation gives you a more accurate estimate compared to a single train-test split, the true test accuracy remains the gold standard for assessing how well your model will perform in a real-world scenario.

3.What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?


A.The number of iterations (folds) in cross-validation can have an impact on the estimate of your model's performance, but the relationship is not necessarily linear, and there are trade-offs to consider.

Effect on Estimate Accuracy:

More Iterations: Increasing the number of iterations in cross-validation (e.g., using 10-fold or 20-fold instead of 5-fold) can lead to a more accurate estimate of your model's performance. With more iterations, you obtain performance metrics from a larger number of different train-test splits, which can help reduce the impact of randomness in the data splits.

Diminishing Returns: However, there are diminishing returns as you increase the number of iterations. Going from 5-fold to 10-fold, for example, typically provides a noticeable improvement in estimate accuracy. However, increasing from 10-fold to 20-fold may yield smaller improvements and require significantly more computation.

Trade-offs:

Computational Cost: More iterations require more computational resources and time. The trade-off between computational cost and improved estimate accuracy should be considered.

Data Size: If you have a small dataset, using a large number of folds can result in very small training sets in each fold, which might lead to less representative training data for each iteration.

Stability of the Model:

Cross-validation can provide a more stable estimate of your model's performance by averaging over multiple splits. However, extremely high numbers of iterations may lead to models that are overly tailored to the specific dataset splits, potentially reducing the model's ability to generalize to new data.
In practice, the choice of the number of iterations in cross-validation (e.g., 5-fold, 10-fold, etc.) depends on factors like the size of your dataset, the computational resources available, and the trade-offs you are willing to make between accuracy and efficiency. It's common to start with a reasonable number of folds (e.g., 5 or 10) and then adjust if needed based on your specific circumstances.

In summary, increasing the number of iterations in cross-validation can generally provide a better estimate of your model's performance, but the improvement may not be linear, and there are trade-offs to consider, including computational cost and the risk of overfitting to the specific dataset splits. The choice of the number of iterations should be made with these factors in mind.

4.Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?



A.Increasing the number of iterations (folds) in cross-validation can help mitigate the impact of having a very small training or validation dataset to some extent, but it doesn't completely solve the problem. There are limitations to how much you can compensate for a small dataset with more iterations.

Here are some considerations:

Mitigating Overfitting in a Small Training Set:

If you have a small training dataset, using more iterations in cross-validation can help reduce the risk of overfitting. By repeatedly splitting the data into different training and validation sets, you expose your model to different subsets of the data, which can encourage it to learn more robust and generalizable patterns.
Limitations of Small Validation Sets:

While increasing the number of iterations can improve the estimate of model performance, it doesn't address the fundamental issue of having a small validation dataset. A small validation dataset may not provide a reliable assessment of how well your model generalizes to new, unseen data. This limitation remains, regardless of the number of iterations.
Trade-offs:

Increasing the number of iterations in cross-validation can lead to very small training sets in each fold if your overall dataset is small. This can potentially introduce other problems, such as increased model variance due to the limited amount of training data in each fold.
In summary, while increasing the number of iterations in cross-validation can help improve model stability and reduce overfitting in the presence of a small training dataset, it cannot completely compensate for a small validation dataset. It's essential to recognize the limitations of your dataset and consider other strategies, such as data augmentation, feature engineering, or acquiring more data if possible, to address the challenges posed by a small dataset effectively. Cross-validation should be used as a tool to better estimate model performance but should not be solely relied upon when dealing with very small datasets.